## TP1 HPC - travail réalisé par MULAPI TITA Ketsia

#### pour les 2 derniers TP, je compte travailler avec Joseph El Kettaneh comme binôme.

In [16]:
import numpy as np

In [17]:
# initialise une
def init_grid(n):
    grid = np.random.randint(2,size=(n,n))
    return grid

In [18]:
def get_nbneigh(grid,coord):
    irange=len(grid)
    jrange=len(grid[0])
    i=coord[0]
    j=coord[1]
    biinf=max(0,i-1)
    bisup=min(irange-1,i+1)
    bjinf=max(0,j-1)
    bjsup=min(jrange-1,j+1)
    nb_neigh=0
    # loop over neighs to count living cells
    for ii in range(biinf,bisup+1):
        for jj in range(bjinf,bjsup+1):
            if grid[ii][jj] == 1:
                nb_neigh=nb_neigh+1
    # in my count I count current cell itself substract the value
    nb_neigh=nb_neigh-grid[i][j]
    return nb_neigh

In [19]:
def evolution1(grid):
    irange=len(grid)
    jrange=len(grid[0])
    res_grid=[[] for i in range(irange)]
    for j in range(jrange):
        for i in range(irange):
            # by default cell_state is dead
            cell_state=0
            # loop over neighs to count living cells
            nb_neigh=get_nbneigh(grid,(i,j))
            # if 2 neighbors cell state isn't modified
            if nb_neigh==2:
                cell_state=grid[i][j]
            # if 3 neighbors cell state is alive
            elif nb_neigh==3:
                cell_state=1
            # in other case cell state is dead (default state)
            res_grid[i].append(cell_state)
    return res_grid

Estimation des temps de restitution pour 10 x 10 100 x 100 et 1000 x 1000

In [9]:
grid = init_grid(10)
%timeit evolution1(grid)

1.28 ms ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
grid = init_grid(100)
%timeit evolution1(grid)

144 ms ± 6.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
grid = init_grid(1000)
%timeit evolution1(grid)

14.9 s ± 645 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


le temps de restitution réduit les performances au fur et à mesure que l'on test notre code sur une grille beaucoup plus de temps !!

# Profilage

### Utiliser cProfile afin d’identifier les fonctions les plus coûteuses (sur des grilles 2000x2000)

In [20]:
grid = init_grid(2000)

In [21]:
import cProfile
cProfile.run("evolution1(grid)") #important de mettre entre guillemets l'instruction à éxécuter
#%prun -D evol1.prof evolution1(grid)

         32000007 function calls in 71.450 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  4000000   56.788    0.000   62.687    0.000 <ipython-input-18-1f759ff32ce4>:1(get_nbneigh)
        1    7.854    7.854   71.372   71.372 <ipython-input-19-ceec1b2be83c>:1(evolution1)
        1    0.021    0.021    0.021    0.021 <ipython-input-19-ceec1b2be83c>:4(<listcomp>)
        1    0.078    0.078   71.450   71.450 <string>:1(<module>)
        1    0.000    0.000   71.450   71.450 {built-in method builtins.exec}
  8000002    1.173    0.000    1.173    0.000 {built-in method builtins.len}
  8000000    2.467    0.000    2.467    0.000 {built-in method builtins.max}
  8000000    2.260    0.000    2.260    0.000 {built-in method builtins.min}
  4000000    0.810    0.000    0.810    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [23]:
%prun -D evol1.prof evolution1(grid)

 
*** Profile stats marshalled to file 'evol1.prof'. 


l'inlining n'est pas non plus autant optimisé !!

# . Créer une fonction evolution1_corr qui est une intégration du corps de get_nbneigh dans evolution1

In [25]:
def evolution1_corr(grid):
    irange=grid.shape[0]
    jrange=grid.shape[1]
    res_grid=[[] for i in range(irange)]
    for j in range(jrange):
        for i in range(irange):
            # by default cell_state is dead
            cell_state=0
            # loop over neighs to count living cells
            biinf=max(0,i-1)
            bisup=min(irange-1,i+1)
            bjinf=max(0,j-1)
            bjsup=min(jrange-1,j+1)
            nb_neigh=0
            # loop over neighs to count living cells
            for ii in range(biinf,bisup+1):
                for jj in range(bjinf,bjsup+1):
                    if grid[ii][jj] == 1:
                        nb_neigh=nb_neigh+1
            # in my count I count current cell itself substract the value
            nb_neigh=nb_neigh-grid[i][j]
            # if 2 neighbors cell state isn't modified
            if nb_neigh==2:
                cell_state=grid[i][j]
            # if 3 neighbors cell state is alive
            elif nb_neigh==3:
                cell_state=1
            # in other case cell state is dead (default state)
            res_grid[i].append(cell_state)
    return res_grid

### Utiliser de nouveau cProfile afin d’estimer un potentiel gain

In [26]:
cProfile.run("evolution1_corr(grid)") #important de mettre entre guillemets l'instruction à éxécuter
#%prun -D evol1.prof evolution1(grid)

         20000005 function calls in 62.884 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   57.805   57.805   62.809   62.809 <ipython-input-25-4661d96cf506>:1(evolution1_corr)
        1    0.000    0.000    0.000    0.000 <ipython-input-25-4661d96cf506>:4(<listcomp>)
        1    0.074    0.074   62.884   62.884 <string>:1(<module>)
        1    0.000    0.000   62.884   62.884 {built-in method builtins.exec}
  8000000    2.225    0.000    2.225    0.000 {built-in method builtins.max}
  8000000    2.120    0.000    2.120    0.000 {built-in method builtins.min}
  4000000    0.658    0.000    0.658    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# À partir de evolution1_corr, écrire une fonction evolution1_ndarray qui modifie la structure de données de res_grid afin d’utiliser un ndarray 2D

In [27]:
def evolution1_ndarray(grid):
    irange=grid.shape[0]
    jrange=grid.shape[1]
    res_grid=np.zeros((irange,jrange))
            
        # à compléter ...
        
        res_grid[i,j]=cell_state
    return res_grid

# Quels impacts sur les performances du code ?

In [ ]:
print("sur une grille 10 x 10")

%timeit evolution1_ndarray(10)

print("analyse de la focntion")

%timeit evolution1_ndarray(grid)

%time evolution1_ndarray(grid)

cProfile.run("evolution1_ndarray(grid)") #important de mettre entre guillemets l'instruction à éxécuter

%prun -D evol1.prof evolution1_ndarray(grid)